# ✨ Feature Engineering

---

## 1. Calendar Features (ฟีเจอร์ปฏิทิน)
- **ภาษาไทย:**  
  เราสร้างฟีเจอร์ใหม่จากวันที่ เช่น `Year`, `Month`, `Week`, `Quarter`, และ `DayOfWeek`  
  เพื่อให้โมเดลเรียนรู้ผลกระทบจากฤดูกาลและช่วงเวลา  

- **English:**  
  We generated new calendar features such as `Year`, `Month`, `Week`, `Quarter`, and `DayOfWeek`.  
  These help the model capture seasonality and time-related effects.  

---


### 1) Load CSV & Parse Dates — โหลดไฟล์และแปลงวันที่

**ไทย:** โหลดข้อมูลด้วย `pd.read_csv(...)` และแปลงคอลัมน์ `Date` เป็นชนิดวันที่ด้วย `pd.to_datetime(..., dayfirst=True, errors='coerce')` จากนั้นเรียงลำดับตาม `Store`, `Date` เพื่อให้ข้อมูลเป็นลำดับเวลา

**English:** Read the dataset using `pd.read_csv(...)` and convert the `Date` column with `pd.to_datetime(..., dayfirst=True, errors='coerce')`. Then sort by `Store`, `Date` to establish temporal order.


In [1]:
import pandas as pd
from pathlib import Path

# ระบุ path dataset
base_dir = Path(r"S:\BusinessAnalyticProject\data")
data_path = base_dir / "Walmart_Sales.csv"

# โหลด dataset
df = pd.read_csv(data_path)

# แปลงวันที่ (ให้ pandas เดา format โดยสมมติ dayfirst)
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

# ตรวจ NaT เผื่อมีวันที่ที่ parse ไม่ได้
print("Unparsed dates (NaT):", df['Date'].isna().sum())

# จัดเรียงตาม Store + Date
df = df.sort_values(['Store', 'Date']).reset_index(drop=True)

df.head()


Unparsed dates (NaT): 0


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106


### 2) Calendar Features — ฟีเจอร์ปฏิทิน

**ไทย:** สร้างฟีเจอร์เวลา เช่น `Year`, `Month`, `Week`, `Quarter`, `DayOfWeek` เพื่อให้โมเดลสามารถเรียนรู้รูปแบบฤดูกาล (seasonality) และความแตกต่างตามวัน/สัปดาห์ได้"

**English:** Create calendar features such as `Year`, `Month`, `Week`, `Quarter`, and `DayOfWeek` to capture seasonality and time-related effects.


In [2]:
df['Year']      = df['Date'].dt.year
df['Month']     = df['Date'].dt.month
df['Week']      = df['Date'].dt.isocalendar().week.astype(int)
df['Quarter']   = df['Date'].dt.quarter
df['DayOfWeek'] = df['Date'].dt.dayofweek  # Monday=0

# seasonality แบบวงจร
import numpy as np
df['Month_sin'] = np.sin(2*np.pi*df['Month']/12)
df['Month_cos'] = np.cos(2*np.pi*df['Month']/12)
df['Peak_Season_Flag'] = df['Month'].isin([11, 12]).astype(int)

pd.options.display.float_format = '{:.4f}'.format  # แสดงทศนิยม 4 ตำแหน่ง
df.head()


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Quarter,DayOfWeek,Month_sin,Month_cos,Peak_Season_Flag
0,1,2010-02-05,1643690.9000,0,42.3100,2.5720,211.0964,8.1060,2010,2,5,1,4,0.8660,0.5000,0
1,1,2010-02-12,1641957.4400,1,38.5100,2.5480,211.2422,8.1060,2010,2,6,1,4,0.8660,0.5000,0
2,1,2010-02-19,1611968.1700,0,39.9300,2.5140,211.2891,8.1060,2010,2,7,1,4,0.8660,0.5000,0
3,1,2010-02-26,1409727.5900,0,46.6300,2.5610,211.3196,8.1060,2010,2,8,1,4,0.8660,0.5000,0
4,1,2010-03-05,1554806.6800,0,46.5000,2.6250,211.3501,8.1060,2010,3,9,1,4,1.0000,0.0000,0


### 3) CPI Deflation → Real Sales — ปรับเงินเฟ้อเป็นยอดขายแท้จริง  

**ภาษาไทย:**  
เราปรับยอดขายรายสัปดาห์จากมูลค่าที่รวมผลของเงินเฟ้อ (nominal) ให้เป็น **มูลค่าคงที่** (real) ด้วย CPI ปีฐาน  
- หา `base_year` จากปีที่เล็กที่สุดในข้อมูล และคำนวณ `base_cpi` เป็นค่า CPI เฉลี่ยของปีนั้น  
- สูตรที่ใช้:  
  \[
  \text{Weekly\_Sales\_Real} = \text{Weekly\_Sales} \times \left(\frac{\text{base\_cpi}}{\text{CPI}}\right)
  \]  
- ทำให้สามารถ **เปรียบเทียบข้ามปีได้อย่างยุติธรรม** เพราะตัดผลของเงินเฟ้อออกไปแล้ว  

**English:**  
We convert nominal weekly sales to **inflation-adjusted (real)** values using a base-year CPI.  
- Determine `base_year` from the earliest year and compute `base_cpi` as the average CPI for that year.  
- Formula:  
  \[
  \text{Weekly\_Sales\_Real} = \text{Weekly\_Sales} \times \left(\frac{\text{base\_cpi}}{\text{CPI}}\right)
  \]  
- This enables **fair cross-year comparisons** by removing inflation effects.  


In [3]:

# ใช้ปีแรกในชุดข้อมูลเป็นปีฐาน
base_year = df['Year'].min()
base_cpi  = df.loc[df['Year'] == base_year, 'CPI'].mean()

df['Weekly_Sales_Real'] = df['Weekly_Sales'] * (base_cpi / df['CPI'])
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Quarter,DayOfWeek,Month_sin,Month_cos,Peak_Season_Flag,Weekly_Sales_Real
0,1,2010-02-05,1643690.9000,0,42.3100,2.5720,211.0964,8.1060,2010,2,5,1,4,0.8660,0.5000,0,1308915.8473
1,1,2010-02-12,1641957.4400,1,38.5100,2.5480,211.2422,8.1060,2010,2,6,1,4,0.8660,0.5000,0,1306632.9090
2,1,2010-02-19,1611968.1700,0,39.9300,2.5140,211.2891,8.1060,2010,2,7,1,4,0.8660,0.5000,0,1282482.9397
3,1,2010-02-26,1409727.5900,0,46.6300,2.5610,211.3196,8.1060,2010,2,8,1,4,0.8660,0.5000,0,1121418.3219
4,1,2010-03-05,1554806.6800,0,46.5000,2.6250,211.3501,8.1060,2010,3,9,1,4,1.0000,0.0000,0,1236648.1954


### 4) Lag & Rolling (Sales) — ฟีเจอร์จากอดีตและค่าเฉลี่ยเคลื่อนที่ (ยอดขาย)

**ภาษาไทย:**  
เราสร้างฟีเจอร์จากยอดขายจริง (`Weekly_Sales_Real`) ต่อ **สาขา (Store)** ด้วยขั้นตอน:  
1. **เรียงข้อมูล** ตาม `Store`, `Date` เพื่อคงลำดับเวลา  
2. **Lag features**: ใช้ค่าจากอดีตเพื่อช่วยทำนายอนาคต เช่น `lag1`, `lag2`, `lag4`, `lag8`, `lag52` (หน่วย = สัปดาห์)  
3. **Rolling features**: ค่าเฉลี่ย/ส่วนเบี่ยงเบนมาตรฐานจากหน้าต่างเลื่อน เช่น `rollmean4`, `rollmean8`, `rollstd8`, `rollstd12`, `rollstd52`  
4. **สำคัญมาก:** ทำ `shift(1)` **ก่อน** rolling เพื่อหลีกเลี่ยงการใช้ข้อมูลอนาคต (data leakage)  
5. ดำเนินการ **แยกตามสาขา** (`group='Store'`) เพื่อไม่ให้ข้อมูลของคนละสาขาปนกัน  

> 📝 **คาดหวัง NaN:** แถวต้น ๆ ของแต่ละสาขาจะมีค่า `NaN` ในคอลัมน์ lag/rolling ตามขนาดหน้าต่าง—ถือว่าเป็นเรื่องปกติ เพราะยังไม่มีข้อมูลย้อนหลังพอ  

**English:**  
We engineer features from `Weekly_Sales_Real` **within each store**:  
1. **Sort** by `Store`, `Date` to preserve chronology  
2. **Lag features**: past values as predictors (e.g., `lag1`, `lag2`, `lag4`, `lag8`, `lag52`)  
3. **Rolling features**: moving mean/std over sliding windows (e.g., `rollmean4`, `rollmean8`, `rollstd8`, `rollstd12`, `rollstd52`)  
4. **Critical:** apply `shift(1)` **before** rolling to avoid future leakage  
5. Operations are **per store** (`group='Store'`) to keep series independent  

**ตัวอย่างคอลัมน์ที่ได้ / Example columns:**  
- `Weekly_Sales_Real_lag8`, `Weekly_Sales_Real_lag52`  
- `Weekly_Sales_Real_rollmean8`, `Weekly_Sales_Real_rollstd8`  

**การตีความ / Interpretation:**  
- `*_lag8` = ยอดขายจริงเมื่อ 8 สัปดาห์ก่อน (ของสาขาเดียวกัน)  
- `*_rollmean8` = ค่าเฉลี่ยยอดขายจริงช่วง 8 สัปดาห์ล่าสุด (ก่อนหน้าจุดเวลา)  
- `*_rollstd8` = ความผันผวนของยอดขายจริงช่วง 8 สัปดาห์ล่าสุด  


In [4]:
def add_lag_roll(d, col='Weekly_Sales_Real', group='Store'):
    d = d.sort_values(['Store','Date']).copy()
    # lags: 1–2 สัปดาห์, 4–8 สัปดาห์, 52 สัปดาห์ (1 ปี)
    for l in [1, 2, 4, 8, 52]:
        d[f'{col}_lag{l}'] = d.groupby(group)[col].shift(l)
    # rolling: ค่าเฉลี่ย/ส่วนเบี่ยงเบนมาตรฐาน
    for w in [4, 8, 12, 52]:
        d[f'{col}_rollmean{w}'] = d.groupby(group)[col].shift(1).rolling(w).mean()
        d[f'{col}_rollstd{w}']  = d.groupby(group)[col].shift(1).rolling(w).std()
    return d

df = add_lag_roll(df, col='Weekly_Sales_Real', group='Store')
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,...,Weekly_Sales_Real_lag8,Weekly_Sales_Real_lag52,Weekly_Sales_Real_rollmean4,Weekly_Sales_Real_rollstd4,Weekly_Sales_Real_rollmean8,Weekly_Sales_Real_rollstd8,Weekly_Sales_Real_rollmean12,Weekly_Sales_Real_rollstd12,Weekly_Sales_Real_rollmean52,Weekly_Sales_Real_rollstd52
0,1,2010-02-05,1643690.9000,0,42.3100,2.5720,211.0964,8.1060,2010,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2010-02-12,1641957.4400,1,38.5100,2.5480,211.2422,8.1060,2010,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2010-02-19,1611968.1700,0,39.9300,2.5140,211.2891,8.1060,2010,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2010-02-26,1409727.5900,0,46.6300,2.5610,211.3196,8.1060,2010,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2010-03-05,1554806.6800,0,46.5000,2.6250,211.3501,8.1060,2010,3,...,NaN,NaN,1254862.5045,89762.9786,NaN,NaN,NaN,NaN,NaN,NaN


### 5) External Drivers (Lag & Rolling) — ตัวแปรภายนอก (Lag และค่าเฉลี่ยเคลื่อนที่)

**ภาษาไทย:**  
เราสร้าง lag และ rolling features สำหรับตัวแปรภายนอก เช่น `Temperature`, `Fuel_Price`, `Unemployment`, และ `CPI`  
- `*_lag1` = ค่าของตัวแปรจากสัปดาห์ก่อนหน้า  
- `*_roll8` = ค่าเฉลี่ยเคลื่อนที่ 8 สัปดาห์ (ย้อนหลัง ไม่รวมปัจจุบัน)  

สิ่งนี้ช่วยให้โมเดลเข้าใจผลกระทบของ **สภาพแวดล้อมทางเศรษฐกิจและฤดูกาล** ต่อยอดขาย

**English:**  
We engineered lag and rolling features for external drivers: `Temperature`, `Fuel_Price`, `Unemployment`, and `CPI`.  
- `*_lag1` = value from one week ago  
- `*_roll8` = 8-week moving average (past only, excluding current)  

This helps the model capture **economic and seasonal impacts** on sales.


In [5]:
for col in ['Temperature', 'Fuel_Price', 'Unemployment', 'CPI']:
    df[f'{col}_lag1']  = df.groupby('Store')[col].shift(1)
    df[f'{col}_roll8'] = df.groupby('Store')[col].shift(1).rolling(8).mean()
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,...,Weekly_Sales_Real_rollmean52,Weekly_Sales_Real_rollstd52,Temperature_lag1,Temperature_roll8,Fuel_Price_lag1,Fuel_Price_roll8,Unemployment_lag1,Unemployment_roll8,CPI_lag1,CPI_roll8
0,1,2010-02-05,1643690.9000,0,42.3100,2.5720,211.0964,8.1060,2010,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2010-02-12,1641957.4400,1,38.5100,2.5480,211.2422,8.1060,2010,2,...,NaN,NaN,42.3100,NaN,2.5720,NaN,8.1060,NaN,211.0964,NaN
2,1,2010-02-19,1611968.1700,0,39.9300,2.5140,211.2891,8.1060,2010,2,...,NaN,NaN,38.5100,NaN,2.5480,NaN,8.1060,NaN,211.2422,NaN
3,1,2010-02-26,1409727.5900,0,46.6300,2.5610,211.3196,8.1060,2010,2,...,NaN,NaN,39.9300,NaN,2.5140,NaN,8.1060,NaN,211.2891,NaN
4,1,2010-03-05,1554806.6800,0,46.5000,2.6250,211.3501,8.1060,2010,3,...,NaN,NaN,46.6300,NaN,2.5610,NaN,8.1060,NaN,211.3196,NaN


### 6) Expanding Features & Momentum — ฟีเจอร์สะสมและโมเมนตัม

**ภาษาไทย:**  
1. `Store_avg_to_date`: ค่าเฉลี่ยสะสมของยอดขายจริง (`Weekly_Sales_Real`) ตั้งแต่ต้นจนถึงปัจจุบัน (per store)  
   → ทำให้เห็นแนวโน้มยอดขายเฉลี่ยสะสมของแต่ละสาขา  
2. `Momentum_8w`: โมเมนตัมหรือค่าเฉลี่ยเคลื่อนที่ 8 สัปดาห์ล่าสุดของยอดขายจริง  
   → ใช้สะท้อนแรงส่ง (trend momentum) ของยอดขาย  

**English:**  
1. `Store_avg_to_date`: cumulative average of `Weekly_Sales_Real` from the beginning up to the current point (per store).  
   → Captures long-term trend of average store sales.  
2. `Momentum_8w`: 8-week moving average of `Weekly_Sales_Real`.  
   → Represents sales momentum over recent weeks.


In [6]:
# ค่าเฉลี่ยสะสมจนถึงสัปดาห์ก่อนหน้า
df['Store_avg_to_date'] = (
    df.groupby('Store')['Weekly_Sales_Real']
      .apply(lambda s: s.shift(1).expanding().mean())
      .reset_index(level=0, drop=True)
)

# โมเมนตัม 8 สัปดาห์ (≥1 = เหนือค่าเฉลี่ยระยะสั้น)
df['Momentum_8w'] = df['Weekly_Sales_Real'] / df['Weekly_Sales_Real'].rolling(8).mean()
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,...,Temperature_lag1,Temperature_roll8,Fuel_Price_lag1,Fuel_Price_roll8,Unemployment_lag1,Unemployment_roll8,CPI_lag1,CPI_roll8,Store_avg_to_date,Momentum_8w
0,1,2010-02-05,1643690.9000,0,42.3100,2.5720,211.0964,8.1060,2010,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2010-02-12,1641957.4400,1,38.5100,2.5480,211.2422,8.1060,2010,2,...,42.3100,NaN,2.5720,NaN,8.1060,NaN,211.0964,NaN,1308915.8473,NaN
2,1,2010-02-19,1611968.1700,0,39.9300,2.5140,211.2891,8.1060,2010,2,...,38.5100,NaN,2.5480,NaN,8.1060,NaN,211.2422,NaN,1307774.3781,NaN
3,1,2010-02-26,1409727.5900,0,46.6300,2.5610,211.3196,8.1060,2010,2,...,39.9300,NaN,2.5140,NaN,8.1060,NaN,211.2891,NaN,1299343.8987,NaN
4,1,2010-03-05,1554806.6800,0,46.5000,2.6250,211.3501,8.1060,2010,3,...,46.6300,NaN,2.5610,NaN,8.1060,NaN,211.3196,NaN,1254862.5045,NaN


### 7) Outlier Capping (IQR) — จำกัดค่าผิดปกติด้วย IQR

**ภาษาไทย:**  
เพื่อป้องกันไม่ให้ยอดขายที่สูง/ต่ำผิดปกติ (outliers) มีผลเกินไป เราใช้ **Interquartile Range (IQR)**:  
- คำนวณ Q1 (25th percentile), Q3 (75th percentile), และ IQR = Q3 - Q1  
- จำกัดค่าที่อยู่นอกช่วง `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]` ให้อยู่ในขอบเขตนี้  
- ค่าที่ได้เก็บในฟีเจอร์ `Weekly_Sales_Real_capped`  

**English:**  
To reduce the influence of extreme outliers, we applied **IQR-based capping**:  
- Compute Q1, Q3, and IQR = Q3 - Q1  
- Cap values outside `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]` within these bounds  
- Resulting feature stored in `Weekly_Sales_Real_capped`  


In [7]:
def iqr_cap(s, k=1.5):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - k*iqr, q3 + k*iqr
    return s.clip(lo, hi)

df['Weekly_Sales_Real_capped'] = df.groupby('Store')['Weekly_Sales_Real'].transform(iqr_cap)
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,...,Temperature_roll8,Fuel_Price_lag1,Fuel_Price_roll8,Unemployment_lag1,Unemployment_roll8,CPI_lag1,CPI_roll8,Store_avg_to_date,Momentum_8w,Weekly_Sales_Real_capped
0,1,2010-02-05,1643690.9000,0,42.3100,2.5720,211.0964,8.1060,2010,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1308915.8473
1,1,2010-02-12,1641957.4400,1,38.5100,2.5480,211.2422,8.1060,2010,2,...,NaN,2.5720,NaN,8.1060,NaN,211.0964,NaN,1308915.8473,NaN,1306632.9090
2,1,2010-02-19,1611968.1700,0,39.9300,2.5140,211.2891,8.1060,2010,2,...,NaN,2.5480,NaN,8.1060,NaN,211.2422,NaN,1307774.3781,NaN,1282482.9397
3,1,2010-02-26,1409727.5900,0,46.6300,2.5610,211.3196,8.1060,2010,2,...,NaN,2.5140,NaN,8.1060,NaN,211.2891,NaN,1299343.8987,NaN,1121418.3219
4,1,2010-03-05,1554806.6800,0,46.5000,2.6250,211.3501,8.1060,2010,3,...,NaN,2.5610,NaN,8.1060,NaN,211.3196,NaN,1254862.5045,NaN,1236648.1954


### 8) Target Variable — สร้างตัวแปรเป้าหมาย

**ภาษาไทย:**  
สร้างคอลัมน์ `Target_next_week` โดยเลื่อนค่า (`shift(-1)`) ของ `Weekly_Sales_Real_capped` ย้อนขึ้นไป 1 สัปดาห์  
→ หมายถึง **ยอดขายจริงของสัปดาห์ถัดไป** ที่เราต้องการให้โมเดลพยากรณ์  

จากนั้นลบแถวที่มีค่า NaN หลังการ shift เพื่อให้ข้อมูลสะอาด

**English:**  
We created the column `Target_next_week` by shifting `Weekly_Sales_Real_capped` by -1.  
→ This represents the **actual sales of the following week**, which becomes our prediction target.  

Finally, we removed NaN rows created by the shift to clean the dataset.


In [8]:
df['Target_next_week'] = df.groupby('Store')['Weekly_Sales_Real_capped'].shift(-1)

# ตัดแถวที่มี NaN จากการทำ lag/shift
before = len(df)
df_model = df.dropna().reset_index(drop=True)
print(f"Rows: {before:,} → {len(df_model):,} after dropna")
df.head()

Rows: 6,435 → 4,050 after dropna


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,...,Fuel_Price_lag1,Fuel_Price_roll8,Unemployment_lag1,Unemployment_roll8,CPI_lag1,CPI_roll8,Store_avg_to_date,Momentum_8w,Weekly_Sales_Real_capped,Target_next_week
0,1,2010-02-05,1643690.9000,0,42.3100,2.5720,211.0964,8.1060,2010,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1308915.8473,1306632.9090
1,1,2010-02-12,1641957.4400,1,38.5100,2.5480,211.2422,8.1060,2010,2,...,2.5720,NaN,8.1060,NaN,211.0964,NaN,1308915.8473,NaN,1306632.9090,1282482.9397
2,1,2010-02-19,1611968.1700,0,39.9300,2.5140,211.2891,8.1060,2010,2,...,2.5480,NaN,8.1060,NaN,211.2422,NaN,1307774.3781,NaN,1282482.9397,1121418.3219
3,1,2010-02-26,1409727.5900,0,46.6300,2.5610,211.3196,8.1060,2010,2,...,2.5140,NaN,8.1060,NaN,211.2891,NaN,1299343.8987,NaN,1121418.3219,1236648.1954
4,1,2010-03-05,1554806.6800,0,46.5000,2.6250,211.3501,8.1060,2010,3,...,2.5610,NaN,8.1060,NaN,211.3196,NaN,1254862.5045,NaN,1236648.1954,1144804.4746


### 9) Feature Set & Meta Info — ฟีเจอร์และข้อมูลประกอบ

**ภาษาไทย:**  
เรารวมชุดฟีเจอร์ที่สร้างไว้ทั้งหมด เช่น  
- Calendar features: `Year`, `Month`, `Week`, `Quarter`, `DayOfWeek`, `Month_sin`, `Month_cos`  
- Flags: `Holiday_Flag`, `Peak_Season_Flag`  
- Lags/Rolling: `Weekly_Sales_Real_lag*`, `Weekly_Sales_Real_roll*`  
- External drivers: `Temperature_lag1`, `Fuel_Price_lag1`, `Unemployment_lag1`, `CPI_lag1`, etc.  
- Store-level: `Store_avg_to_date`, `Momentum_8w`  

และกำหนด `Target_next_week` เป็น y เพื่อใช้ train model  
ส่วน `meta` เก็บคอลัมน์ประกอบ เช่น `Store`, `Date`, `Weekly_Sales`, `Weekly_Sales_Real` ไว้อ้างอิงภายหลัง

**English:**  
We consolidated all engineered features, including:  
- Calendar: `Year`, `Month`, `Week`, `Quarter`, `DayOfWeek`, `Month_sin`, `Month_cos`  
- Flags: `Holiday_Flag`, `Peak_Season_Flag`  
- Lags/Rolling: `Weekly_Sales_Real_lag*`, `Weekly_Sales_Real_roll*`  
- External drivers: `Temperature_lag1`, `Fuel_Price_lag1`, `Unemployment_lag1`, `CPI_lag1`, etc.  
- Store-level: `Store_avg_to_date`, `Momentum_8w`  

We then defined `Target_next_week` as `y` for training,  
while `meta` keeps supporting info (`Store`, `Date`, `Weekly_Sales`, `Weekly_Sales_Real`) for reference.


In [10]:
feature_cols = [
    # calendar
    'Year','Month','Week','Quarter','DayOfWeek','Month_sin','Month_cos',
    # flags
    'Holiday_Flag','Peak_Season_Flag',
    # lags/rolls ของยอดขายจริง
    'Weekly_Sales_Real_lag1','Weekly_Sales_Real_lag2','Weekly_Sales_Real_lag4',
    'Weekly_Sales_Real_rollmean4','Weekly_Sales_Real_rollmean8','Weekly_Sales_Real_rollstd8',
    # external drivers
    'Temperature_lag1','Fuel_Price_lag1','Unemployment_lag1','CPI_lag1',
    'Temperature_roll8','Fuel_Price_roll8','Unemployment_roll8',
    # store
    'Store_avg_to_date','Momentum_8w'
]

X = df_model[feature_cols].copy()
y = df_model['Target_next_week'].copy()

meta = df_model[['Store','Date','Weekly_Sales','Weekly_Sales_Real']]
X.shape, y.shape
df.head()


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,...,Fuel_Price_lag1,Fuel_Price_roll8,Unemployment_lag1,Unemployment_roll8,CPI_lag1,CPI_roll8,Store_avg_to_date,Momentum_8w,Weekly_Sales_Real_capped,Target_next_week
0,1,2010-02-05,1643690.9000,0,42.3100,2.5720,211.0964,8.1060,2010,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1308915.8473,1306632.9090
1,1,2010-02-12,1641957.4400,1,38.5100,2.5480,211.2422,8.1060,2010,2,...,2.5720,NaN,8.1060,NaN,211.0964,NaN,1308915.8473,NaN,1306632.9090,1282482.9397
2,1,2010-02-19,1611968.1700,0,39.9300,2.5140,211.2891,8.1060,2010,2,...,2.5480,NaN,8.1060,NaN,211.2422,NaN,1307774.3781,NaN,1282482.9397,1121418.3219
3,1,2010-02-26,1409727.5900,0,46.6300,2.5610,211.3196,8.1060,2010,2,...,2.5140,NaN,8.1060,NaN,211.2891,NaN,1299343.8987,NaN,1121418.3219,1236648.1954
4,1,2010-03-05,1554806.6800,0,46.5000,2.6250,211.3501,8.1060,2010,3,...,2.5610,NaN,8.1060,NaN,211.3196,NaN,1254862.5045,NaN,1236648.1954,1144804.4746


### 10) Export Processed Dataset — ส่งออกข้อมูลที่ประมวลผลแล้ว

**ภาษาไทย:**  
เราได้รวม `meta`, `X`, และ `y` เข้าด้วยกัน และบันทึกเป็นไฟล์ CSV (`walmart_features_weekly.csv`)  
โดยเก็บไว้ในโฟลเดอร์ `data/processed` เพื่อให้ใช้ในขั้นตอนการสร้างโมเดลต่อไปได้สะดวก

**English:**  
We concatenated `meta`, `X`, and `y` into a single DataFrame and exported it as a CSV file (`walmart_features_weekly.csv`).  
The file is saved under `data/processed` for easy reuse during the modeling stage.


In [11]:
from pathlib import Path
out_dir = Path(r"S:\BusinessAnalyticProject\data\processed")
out_dir.mkdir(parents=True, exist_ok=True)

df_out = pd.concat([meta, X, y.rename('Target')], axis=1)
df_out.to_csv(out_dir / "walmart_features_weekly.csv", index=False)

df_out.head()


,Store,Date,Weekly_Sales,Weekly_Sales_Real,Year,Month,Week,Quarter,DayOfWeek,Month_sin,...,Temperature_lag1,Fuel_Price_lag1,Unemployment_lag1,CPI_lag1,Temperature_roll8,Fuel_Price_roll8,Unemployment_roll8,Store_avg_to_date,Momentum_8w,Target
0,1,2011-02-04,1606629.5800,1270552.0874,2011,2,5,1,4,0.8660,...,43.8300,3.0100,7.7420,212.1971,46.0588,2.9407,7.7900,1204743.5771,1.0052,1302279.9853
1,1,2011-02-11,1649614.9300,1302279.9853,2011,2,6,1,4,0.8660,...,42.2700,2.9890,7.7420,212.5669,45.5512,2.9590,7.7780,1205985.2471,1.0513,1329726.0887
2,1,2011-02-18,1686842.7800,1329726.0887,2011,2,7,1,4,0.8660,...,36.3900,3.0220,7.7420,212.9367,43.8700,2.9781,7.7660,1207768.4830,1.1388,1146837.8671
3,1,2011-02-25,1456800.2800,1146837.8671,2011,2,8,1,4,0.8660,...,57.3600,3.0450,7.7420,213.2479,44.4988,2.9980,7.7540,1209985.8940,0.9760,1286383.4399
4,1,2011-03-04,1636263.4100,1286383.4399,2011,3,9,1,4,1.0000,...,62.9000,3.0650,7.7420,213.5356,46.3075,3.0132,7.7420,1208858.2507,1.0789,1219433.8976


## 📌 Conclusion — Feature Engineering Summary

**ภาษาไทย:**  
ในขั้นตอน Feature Engineering เราได้สร้างและปรับปรุงฟีเจอร์หลายประเภทเพื่อให้โมเดลสามารถเรียนรู้ได้ดีขึ้น ได้แก่:  
- **Calendar Features:** เช่น Year, Month, Week, Quarter, DayOfWeek เพื่อเก็บบริบทของเวลา  
- **Seasonality Encoding:** ใช้ sine/cos เพื่อแทนรูปแบบวงจรของเดือน  
- **Lag & Rolling Features:** เพิ่มค่าล่าช้า (lag) และค่าเฉลี่ย/ส่วนเบี่ยงเบนมาตรฐาน (rolling mean/std) เพื่อสะท้อนแนวโน้มและความผันผวนในอดีต  
- **External Drivers:** เช่น Temperature, Fuel Price, CPI, และ Unemployment เพื่อวัดผลกระทบจากปัจจัยภายนอก  
- **Target Variable:** สร้าง `Target_next_week` สำหรับการพยากรณ์ยอดขายในอนาคต  
- **Export Dataset:** รวมข้อมูลทั้งหมดและบันทึกไว้เป็นไฟล์ CSV (`walmart_features_weekly.csv`) สำหรับใช้ในโมเดล Machine Learning  

ผลลัพธ์คือชุดข้อมูลที่สะอาดและพร้อมใช้งานในการเทรนโมเดลพยากรณ์ยอดขายต่อไป 🚀

---

**English:**  
In the Feature Engineering step, we created and refined multiple types of features to enhance the model’s learning ability:  
- **Calendar Features:** Year, Month, Week, Quarter, DayOfWeek to capture temporal context  
- **Seasonality Encoding:** sine/cos transformations to represent cyclical seasonal patterns  
- **Lag & Rolling Features:** lag values and rolling mean/std to reflect historical trends and volatility  
- **External Drivers:** Temperature, Fuel Price, CPI, and Unemployment as external influencing factors  
- **Target Variable:** generated `Target_next_week` for forecasting future sales  
- **Export Dataset:** combined all data and saved it as a CSV (`walmart_features_weekly.csv`) for downstream ML models  

The result is a clean and enriched dataset, ready for building predictive models for Walmart’s weekly sales 📊✨

